# City Raininess

##### This notebook allows a user to enter a city and country, and will locate the latitude and longitude of the most populated city following those parameters,if there is one; this utilizes the world_cities CSV. Then, based on the latitude and longitude of the city, historical and forecast data will be determined from the OpenMeteo API. A raininess index is defined, weighting recent raininess more heavily than less recent data.

***

In [1]:
# Importing standard libraries
import pandas as pd
import requests
import datetime
import pprint
import math
import lets_plot
import time

# Importing custom libraries
from Analysis_Functions import *

***

### 1. Converting City and Country to Latitude and Longitude

***

### 2. Collecting Historical Data (Last 2 Years)

***

### 3. Collecting Forecast Data (Next 7 Days)

***

### 4. Defining the Raininess Index

##### The raininess index combines the daily historical data of amount of rain (mm), hours of precipitation, and the mean probability of precipitation with the hourly forecast data of amount of precipitation

#### Collecting London data:

##### To collect data for a particular city (in this case, London), we can call the functions defined about to gather both historical and forecast data

In [2]:
# Collecting historical and forecast precipitation data for London, GB
city = 'London'
country = 'GB'
latitude, longitude = get_city_latlon(city, country)
historical_precipitation = f"   Historical precipitation: {get_historical_precipitation(latitude, longitude)}"
forecast_precipitation = f"   Forecast precipitation: {get_forecast_precipitation(latitude, longitude)}"
print(f"Weather data for {city}, {country}:")
print(historical_precipitation)
print(forecast_precipitation)

# storing above output into a variable
London = {
    "City": city,
    "Country": country,
    "Historical Precipitation": get_historical_precipitation(latitude, longitude),
    "Forecast Precipitation": get_forecast_precipitation(latitude, longitude)
}

Weather data for London, GB:
   Historical precipitation: {'Historical Rain Sum': [0.3, 12.8, 1.7, 5.0, 1.4, 4.6, 2.3, 4.0, 15.4, 7.7, 0.2, 0.0, 0.0, 0.0, 0.0, 0.9, 1.7, 2.3, 0.0, 1.6, 4.7, 12.3, 2.4, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 5.5, 0.0, 3.6, 1.3, 4.8, 0.1, 7.5, 8.7, 6.6, 3.1, 2.5, 7.9, 6.6, 10.8, 14.5, 5.2, 4.3, 0.0, 1.7, 0.0, 7.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.1, 0.0, 0.0, 1.3, 0.1, 3.5, 6.5, 0.1, 0.0, 1.0, 4.8, 5.4, 12.5, 6.2, 2.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 2.9, 6.9, 1.5, 0.0, 0.6, 0.1, 0.0, 6.7, 0.3, 0.1, 0.0, 0.0, 0.0, 1.0, 18.8, 2.4, 0.3, 0.4, 12.8, 0.3, 6.8, 15.7, 1.0, 2.2, 1.6, 3.9, 0.0, 0.0, 1.2, 4.3, 2.7, 0.8, 7.2, 6.1, 5.5, 0.0, 4.2, 0.2, 5.1, 15.4, 0.0, 0.0, 3.3, 4.0, 3.1, 0.2, 1.6, 0.1, 3.5, 1.7, 0.0, 0.0, 0.6, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.6, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 6.4, 2.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 13.6, 3.4, 7.9, 2.2, 0.0, 1.0, 0.2, 0

#### Converting London Data to Raininess Index

##### Now, a raininess index will be created, which weights more recent weather more heavily to determine how "rainy" a city is

##### The weighting works as follows:

The most emphasis is placed on the past two weeks of rain, with the least recent historical data being weighted the least. 

As forecast data is often not very accurate (e.g., many very rainy cities often show no rain in the forecast), forecast data will not **decrease** the raininess index of a city. However, if there is a substantial amount of rain in the forecast, this will **increase** the raininess index.

#### For now, the raininess index is just a number. 

We are next going to run a number of cities through these functions in order to create a distribution of raininess; from this distribution, the bottom 25% of cities (in terms of raininess index) will be deemed "Not rainy", while the top 25% will be deemed "Very rainy". Our goal here is to see where London falls in this distribution.

***

### 4. Comparing London to Other Cities

To get a wide range of latitudes and logitudes, I will first compare London to other world capitals, contained in the file [country-capital-lat-long-population.csv](https://gist.github.com/ofou/df09a6834a8421b4f376c875194915c9)

Because London is a capital, the London data will be derived from the CSV while the capitals are iterated through.

In [35]:
# Creating a DF of world cities
cities_df = pd.read_csv('country-capital-lat-long-population.csv')

# Initialize an empty list to store raininess index values
raininess_values = []

# Loop through each row in the DataFrame
for index, row in cities_df.iterrows():
    try:
        # Calculate the raininess index for the current city
        raininess_index = raininess({
            "City": row['Capital City'],
            "Country": row['Country'],
            "Historical Precipitation": get_historical_precipitation(row['Latitude'], row['Longitude']),
            "Forecast Precipitation": get_forecast_precipitation(row['Latitude'], row['Longitude'])
        })

        # Add a delay between each request to avoid hitting rate limits
        time.sleep(5)
        
    except KeyError as e:
        # Log an error message and skip to the next iteration
        print(f"KeyError for {row['Capital City']}, {row['Country']}: {e}")
        raininess_index = None  # Optional: assign None or a default value if data is missing
    
    # Append the calculated raininess index (or None) to the list
    raininess_values.append(raininess_index)

cities_df['Raininess'] = raininess_values

# Sort the cities by raininess index
cities_df = cities_df.sort_values(by='Raininess', ascending=False)

# Reset the index and remove the old one
cities_df = cities_df.reset_index(drop=True)

# Set the index to start at 1 instead of 0
cities_df.index = cities_df.index + 1

# Select only the relevant columns to display
sample_cities_df = cities_df[['Capital City', 'Country', 'Latitude', 'Longitude', 'Raininess']]

# Display the top 10 rainiest cities
print("Top 10 Rainiest Capital Cities:")
print(sample_cities_df.head(10))

Top 10 Rainiest Capital Cities:
           Capital City                      Country  Latitude  Longitude  \
1               Cayenne                French Guiana    4.9333   -52.3333   
2                 Koror                        Palau    7.3426   134.4789   
3               Palikir  Micronesia (Fed. States of)    6.9174   158.1588   
4                Majuro             Marshall Islands    7.0897   171.3803   
5               Papeete             French Polynesia  -17.5333  -149.5667   
6   Bandar Seri Begawan            Brunei Darussalam    4.9403   114.9481   
7              Monrovia                      Liberia    6.3005   -10.7969   
8              São Tomé        Sao Tome and Principe    0.3365     6.7273   
9                  Apia                        Samoa  -13.8333  -171.7667   
10             Funafuti                       Tuvalu   -8.5189   179.1991   

    Raininess  
1   72.568900  
2   72.185233  
3   71.760855  
4   71.673915  
5   70.664145  
6   70.391859  
7   70.3